In [8]:
from Moduli.ClassDictionary import ClassDictionary as cDict
from Moduli.Dataset import ScenesDatabase as sd
from Moduli.BellomoClassifier import BellomoClassifier

from torchvision import transforms
import numpy as np
import torch
from torch.utils.data import DataLoader
import tkinter as tk
from PIL import Image, ImageTk
from tkinter import messagebox, Frame

m = [0.485, 0.456, 0.408]
s = [0.229, 0.224, 0.225]

test_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(m, s)
])

test = sd('Bellomo/Training/', 'Bellomo/Training/training.txt', transform=test_transform, split="second")

def transform_input(Dataset):
    loader = DataLoader(Dataset, batch_size=100, num_workers=0, drop_last=True, shuffle=True)
    return loader

class BellomoApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.i = 0
        self.title("Demo")
        self.geometry("260x480")

        # GUI Components

        self.frame = Frame(self, bg="#f5f5f5")
        self.frame.place(relwidth=1, relheight=1)
        self.canvas = tk.Canvas(self.frame, bg="gray", width=224, height=224)
        self.canvas.pack(pady=20)
        self.prediction_label = tk.Label(self.frame, text="", font=('Arial', 12), bg="#f5f5f5")
        self.prediction_label.pack(pady=10)
        self.btn_run_demo = tk.Button(self.frame, text="Run Demo", command=self.call_demo)
        self.btn_run_demo.pack(pady=10)
        self.attributes('-topmost', True)

        # Caricamento del modello
        device = "cuda" if torch.cuda.is_available() else "cpu"
        print(f"Using device: {device}")
        try:
            self.model = BellomoClassifier().eval()
            self.model.load_state_dict(torch.load('%s-%d.pth' % ("weight/BellomoClassifier-5", 55), map_location=torch.device(device)))
            self.model.to(device)
        except Exception as e:
            print(f"Error loading the model: {e}")

    def call_demo(self):
        self.demo(test, self.i)

    def demo(self, dataset, index):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        classes = cDict().get()

        image = dataset[index]['image'].unsqueeze(0).to(device)
        label = dataset[index]['label']

        output = self.model(image)
        pred = output.argmax(dim=1).item()

        # Mostra l'immagine
        img_np = image.squeeze(0).cpu().numpy().transpose(1, 2, 0)
        img_np = s * img_np + m
        img_np = np.clip(img_np, 0, 1)
        self.display_image_on_canvas(img_np)

        # Mostra previsione e label originale
        prediction_text = f"Predicted: {classes[pred]}\nOriginal: {classes[label]}"
        self.display_prediction_text(prediction_text)

        self.i+=1

    def display_image_on_canvas(self, img_np_array):
        img = Image.fromarray((img_np_array * 255).astype(np.uint8))
        photo = ImageTk.PhotoImage(img)
        self.canvas.image = photo
        self.canvas.create_image(0, 0, anchor=tk.NW, image=photo)

    def display_prediction_text(self, prediction_text):
        self.prediction_label.config(text=prediction_text)

if __name__ == '__main__':
    app = BellomoApp()
    app.mainloop()


TclError: can't use "pyimage89" as iconphoto: not a photo image